In [1]:
import os 
import sys 

In [16]:
os.getcwd()

'/home/googlyji/ml_projects/nlp_projects/Resume_Type/Resume_Type'

In [5]:
os.chdir("../")

In [17]:
pd.read_csv("artifacts/data_ingestion/train.csv")

,Unnamed: 0,Category,Resume
0,468,Automation Testing,* Excellent grasping power in learning new con...
1,857,Blockchain,Hobbies â¢ Playing Chess â¢ Solving Rubik's ...
2,549,Operations Manager,IT SKILLS â¢ Well versed with MS Office and I...
3,280,Health and fitness,SKILLS: Computer: â¢ Can easily operate in Op...
4,109,Arts,I Other Skills Course/Skill Name Board Year Of...
...,...,...,...
764,106,Arts,"Additional qualifications April 2000, Web Desi..."
765,270,Health and fitness,Education Details \r\nMay 2014 Diploma Nutriti...
766,860,Blockchain,"SOFTWARE SKILLS: Languages: C, C++ & java Oper..."
767,435,SAP Developer,Education Details \r\nJanuary 2016 Bachelor Of...


In [6]:
import pandas as pd 
import numpy as np 

In [25]:
from sklearn.preprocessing import LabelEncoder

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
from src.utils import * 
from src.constants import * 
import nltk
from dataclasses import dataclass

In [19]:
from src.exception import Custom_Exception

In [8]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /home/googlyji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
import pandas as pd 
import numpy as np 
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /home/googlyji/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
#entity 
@dataclass
class transformation:
  root_dir: Path
  transformed_train_data: Path
  transformed_test_data: Path
  transformation_pickle_file: Path 

In [31]:
# configuration 
class Configuration_Manager:
    def __init__(self,config=config_path,params=params_path):
        self.config=read_yaml(config)
        self.params=read_yaml(params)

        create_directories([self.config.artifacts])

    def trasformation_config(self)-> transformation:
        config=self.config.trasformation

        create_directories([config.root_dir])

        transformation_configuration=transformation(
            root_dir=config.root_dir,
            transformed_train_data=config.transformed_train_data,
            transformed_test_data=config.transformed_test_data,
            transformation_pickle_file=config.transformation_pickle_file
            )
        
        return transformation_configuration
    


In [32]:
# components
class Data_transformation:
    def __init__(self,config:transformation):
        self.config=config

    def remvoing_unwanted_columns(self):
        self.df_train=pd.read_csv("artifacts/data_ingestion/train.csv")
        self.df_train.drop("Unnamed: 0",inplace=True,axis=1)
        self.df_test=pd.read_csv("artifacts/data_ingestion/test.csv")
        self.df_test.drop("Unnamed: 0",inplace=True,axis=1)
    

            
    def comp_transformation(self):
        corpus=[]
        wn=WordNetLemmatizer()
        for i in range(len(self.df_train)):
            sentence = re.sub("[^a-zA-Z]", " ", self.df_train['Resume'][i])
            sentence=sentence.lower()
            sentence=sentence.split()
            sentence=[wn.lemmatize(word) for word in sentence if word not in stopwords.words("english")]
            sentence=" ".join(sentence)
            corpus.append(sentence)
        
        self.df_train.drop("Resume",axis=1,inplace=True)

        self.df_train['Resume']=corpus

        cropus_1=[]
        for i in range(len(self.df_test)):
            sentence = re.sub("[^a-zA-Z]", " ", self.df_test['Resume'][i])
            sentence=sentence.lower()
            sentence=sentence.split()
            sentence=[wn.lemmatize(word) for word in sentence if word not in stopwords.words("english")]
            sentence=" ".join(sentence)
            cropus_1.append(sentence)
        
        self.df_test.drop("Resume",axis=1,inplace=True)

        self.df_test["Resume"]=cropus_1


        self.df_train["Category"]=self.df_train["Category"].map({'Data Science':0, 'HR':1, 'Advocate':2, 'Arts':3, 'Web Designing':4,
       'Mechanical Engineer':5, 'Sales':6, 'Health and fitness':7,
       'Civil Engineer':8, 'Java Developer':9, 'Business Analyst':10,
       'SAP Developer':11, 'Automation Testing':12, 'Electrical Engineering':13,
       'Operations Manager':14, 'Python Developer':15, 'DevOps Engineer':16,
       'Network Security Engineer':17, 'PMO':18, 'Database':19, 'Hadoop':20,
       'ETL Developer':21, 'DotNet Developer':22, 'Blockchain':23, 'Testing':24})

        self.df_test["Category"]=self.df_test["Category"].map({'Data Science':0, 'HR':1, 'Advocate':2, 'Arts':3, 'Web Designing':4,
       'Mechanical Engineer':5, 'Sales':6, 'Health and fitness':7,
       'Civil Engineer':8, 'Java Developer':9, 'Business Analyst':10,
       'SAP Developer':11, 'Automation Testing':12, 'Electrical Engineering':13,
       'Operations Manager':14, 'Python Developer':15, 'DevOps Engineer':16,
       'Network Security Engineer':17, 'PMO':18, 'Database':19, 'Hadoop':20,
       'ETL Developer':21, 'DotNet Developer':22, 'Blockchain':23, 'Testing':24})

        self.df_train.to_csv(self.config.transformed_train_data,index=False,header=True)
        self.df_train.to_csv(self.config.transformed_test_data,index=False,header=True)


    

        

In [33]:
try:
    a=Configuration_Manager()
    v=a.trasformation_config()
    z=Data_transformation(v)
    z.remvoing_unwanted_columns()
    z.comp_transformation()

except Exception as e:
    raise(Custom_Exception(e,sys))
